In [1]:
import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
     

import torchvision
import torchvision.transforms as transforms

from models import *


global best_prec
use_gpu = torch.cuda.is_available()
print('=> Building model...')
    
    
    
batch_size = 128
model_name = "VGG16_quant"
model = VGG16_quant()

=> Building model...


In [2]:
print(model)

VGG_quant(
  (features): Sequential(
    (0): QuantConv2d(
      3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): QuantConv2d(
      64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): QuantConv2d(
      64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): QuantConv2d(
      128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 

In [3]:

normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])


train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


print_freq = 200 # every 100 batches, accuracy printed. Here, each batch includes "batch_size" data points
# CIFAR10 has 50,000 training data, and 10,000 validation data.

def train(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))

            

def validate(val_loader, model, criterion ):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
         
            input, target = input.cuda(), target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:  # This line shows how frequently print out the status. e.g., i%5 => every 5 batch, prints out
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Prec {top1.avg:.3f}% '.format(top1=top1))
    return top1.avg


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

        
def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))


def adjust_learning_rate(optimizer, epoch):
    """For resnet, the lr starts from 0.1, and is divided by 10 at 80 and 120 epochs"""
    adjust_list = [70, 90]
    if epoch in adjust_list:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1        

#model = nn.DataParallel(model).cuda()
#all_params = checkpoint['state_dict']
#model.load_state_dict(all_params, strict=False)
#criterion = nn.CrossEntropyLoss().cuda()
#validate(testloader, model, criterion)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
# This cell won't be given, but students will complete the training

'''lr = 4e-2
weight_decay = 1e-4
epochs = 130
best_prec = 0

#model = nn.DataParallel(model).cuda()
model.cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
#cudnn.benchmark = True

if not os.path.exists('result'):
    os.makedirs('result')
fdir = 'result/'+str(model_name)
if not os.path.exists(fdir):
    os.makedirs(fdir)
        

for epoch in range(0, epochs):
    adjust_learning_rate(optimizer, epoch)

    train(trainloader, model, criterion, optimizer, epoch)
    
    # evaluate on test set
    print("Validation starts")
    prec = validate(testloader, model, criterion)

    # remember best precision and save checkpoint
    is_best = prec > best_prec
    best_prec = max(prec,best_prec)
    print('best acc: {:1f}'.format(best_prec))
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_prec': best_prec,
        'optimizer': optimizer.state_dict(),
    }, is_best, fdir)'''

'lr = 4e-2\nweight_decay = 1e-4\nepochs = 130\nbest_prec = 0\n\n#model = nn.DataParallel(model).cuda()\nmodel.cuda()\ncriterion = nn.CrossEntropyLoss().cuda()\noptimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)\n#cudnn.benchmark = True\n\nif not os.path.exists(\'result\'):\n    os.makedirs(\'result\')\nfdir = \'result/\'+str(model_name)\nif not os.path.exists(fdir):\n    os.makedirs(fdir)\n        \n\nfor epoch in range(0, epochs):\n    adjust_learning_rate(optimizer, epoch)\n\n    train(trainloader, model, criterion, optimizer, epoch)\n    \n    # evaluate on test set\n    print("Validation starts")\n    prec = validate(testloader, model, criterion)\n\n    # remember best precision and save checkpoint\n    is_best = prec > best_prec\n    best_prec = max(prec,best_prec)\n    print(\'best acc: {:1f}\'.format(best_prec))\n    save_checkpoint({\n        \'epoch\': epoch + 1,\n        \'state_dict\': model.state_dict(),\n        \'best_prec\': b

In [5]:
PATH = "result/NVGG16_quant/model_best.pth.tar"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda") 

model.cuda()
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))


Test set: Accuracy: 9213/10000 (92%)



In [6]:
#0,3,7,10,14,17,20,24,27,29,33,,36,,39,
import torch.nn.utils.prune as prune

prune.l1_unstructured(model.features[0], name='weight', amount=0.8)
prune.l1_unstructured(model.features[3], name='weight', amount=0.8)
prune.l1_unstructured(model.features[7], name='weight',   amount=0.8)
prune.l1_unstructured(model.features[10], name='weight',   amount=0.8)

prune.l1_unstructured(model.features[14], name='weight', amount=0.8)
prune.l1_unstructured(model.features[17], name='weight', amount=0.8)
prune.l1_unstructured(model.features[20], name='weight',   amount=0.8)
prune.l1_unstructured(model.features[24], name='weight',   amount=0.8)

prune.l1_unstructured(model.features[27], name='weight', amount=0.8)
prune.l1_unstructured(model.features[29], name='weight', amount=0.8)
prune.l1_unstructured(model.features[33], name='weight',   amount=0.8)
prune.l1_unstructured(model.features[36], name='weight',   amount=0.8)
prune.l1_unstructured(model.features[39], name='weight',   amount=0.8)

QuantConv2d(
  512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
)

In [7]:
### Check sparsity ###
mask1 = model.features[39].weight_mask
sparsity_mask1 = (mask1 == 0).sum() / mask1.nelement()

print("Sparsity level: ", sparsity_mask1)

Sparsity level:  tensor(0.8000, device='cuda:0')


In [8]:
## check accuracy after pruning
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))


Test set: Accuracy: 1000/10000 (10%)



In [9]:
## Start finetuning (training here), and see how much you can recover your accuracy ##
## You can change hyper parameters such as epochs or lr ##
# number of epochs to train the model
n_epochs = 30  
# per epoch, all the training data set is used once
model.train() # prep model for training
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss().to(device)
for epoch in range(n_epochs):
    # monitor training loss
    train_loss = 0.0
    
    ###################
    # train the model #
    ###################
    for data, target in trainloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0) # as loss is tensor, .item() needed to get the value
        
    # print training statistics 
    # calculate average loss over an epoch
    train_loss = train_loss/len(trainloader.dataset)

    print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch+1, train_loss))
    



Epoch: 1 	Training Loss: 1.663813
Epoch: 2 	Training Loss: 0.559841
Epoch: 3 	Training Loss: 0.401627
Epoch: 4 	Training Loss: 0.330964
Epoch: 5 	Training Loss: 0.294967
Epoch: 6 	Training Loss: 0.273961
Epoch: 7 	Training Loss: 0.254914
Epoch: 8 	Training Loss: 0.242336
Epoch: 9 	Training Loss: 0.224879
Epoch: 10 	Training Loss: 0.217362
Epoch: 11 	Training Loss: 0.210652
Epoch: 12 	Training Loss: 0.397709
Epoch: 13 	Training Loss: 0.305775
Epoch: 14 	Training Loss: 0.259167
Epoch: 15 	Training Loss: 0.238323
Epoch: 16 	Training Loss: 0.229755
Epoch: 17 	Training Loss: 0.213996
Epoch: 18 	Training Loss: 0.206446
Epoch: 19 	Training Loss: 0.190781
Epoch: 20 	Training Loss: 0.188687
Epoch: 21 	Training Loss: 0.179383
Epoch: 22 	Training Loss: 0.173283
Epoch: 23 	Training Loss: 0.201890
Epoch: 24 	Training Loss: 0.186723
Epoch: 25 	Training Loss: 0.178331
Epoch: 26 	Training Loss: 0.169284
Epoch: 27 	Training Loss: 0.159815
Epoch: 28 	Training Loss: 0.157490
Epoch: 29 	Training Loss: 0.1

In [10]:
## check accuracy after pruning
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))


Test set: Accuracy: 8827/10000 (88%)



In [11]:
weight_q = model.features[27].weight_q
w_alpha = model.features[27].weight_quant.wgt_alpha
w_bit = 4

weight_int = weight_q / (w_alpha / (2**(w_bit-1)-1))

In [12]:
class SaveOutput:
    def __init__(self):
        self.outputs = []
    def __call__(self, module, module_in):
        self.outputs.append(module_in)
    def clear(self):
        self.outputs = []  
        
######### Save inputs from selected layer ##########
save_output = SaveOutput()
i = 0

for layer in model.modules():
    i = i+1
    if isinstance(layer, QuantConv2d):
        print(i,"-th layer prehooked")
        layer.register_forward_pre_hook(save_output)             
####################################################

dataiter = iter(testloader)
images, labels = next(dataiter)
images = images.to(device)
out = model(images)

3 -th layer prehooked
7 -th layer prehooked
12 -th layer prehooked
16 -th layer prehooked
21 -th layer prehooked
25 -th layer prehooked
29 -th layer prehooked
34 -th layer prehooked
38 -th layer prehooked
41 -th layer prehooked
46 -th layer prehooked
50 -th layer prehooked
54 -th layer prehooked


In [13]:
act = save_output.outputs[8][0]
act_alpha  = model.features[27].act_alpha
act_bit = 4
act_quant_fn = act_quantization(act_bit)

act_q = act_quant_fn(act, act_alpha)

act_int = act_q / (act_alpha / (2**act_bit-1))
print(act_int)

tensor([[[[ 0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  1.0000],
          [ 0.0000,  0.0000,  0.0000,  4.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000]],

         [[ 0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000],
          [ 4.0000,  5.0000,  0.0000,  0.0000],
          [ 2.0000,  0.0000,  0.0000,  0.0000]],

         [[10.0000,  5.0000, 11.0000,  8.0000],
          [ 6.0000,  0.0000,  0.0000,  0.0000],
          [ 6.0000,  0.0000,  0.0000,  0.0000],
          [11.0000,  3.0000,  2.0000,  0.0000]],

         ...,

         [[ 3.0000,  1.0000,  0.0000,  0.0000],
          [ 1.0000,  2.0000,  0.0000,  0.0000],
          [ 2.0000,  2.0000,  1.0000,  1.0000],
          [ 1.0000,  0.0000,  0.0000,  0.0000]],

         [[ 0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000],
          [ 1.0000,  1.0000,  0.0000,  1.0000]],

         [[ 0.0

In [14]:
conv_int = torch.nn.Conv2d(in_channels = 8, out_channels=8, kernel_size = 3, padding=1)
conv_int.weight = torch.nn.parameter.Parameter(weight_int)
conv_int.bias = model.features[27].bias
output_int = conv_int(act_int)
a = nn.ReLU()
output_recovered = a(output_int * (act_alpha / (2**act_bit-1)) * (w_alpha / (2**(w_bit-1)-1)))
print(output_recovered)

tensor([[[[1.9402e+00, 2.6435e+00, 6.0389e+00, 3.9047e+00],
          [4.3655e+00, 5.7721e+00, 8.2459e+00, 3.7592e+00],
          [2.3768e+00, 4.8263e+00, 1.1253e+01, 5.9177e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00]],

         [[1.4552e+00, 3.4196e+00, 3.4196e+00, 0.0000e+00],
          [1.8917e+00, 3.2984e+00, 5.0931e+00, 3.0316e+00],
          [8.7310e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [3.0801e+00, 1.6249e+00, 0.0000e+00, 0.0000e+00]],

         [[1.3096e+00, 8.4885e-01, 7.5183e-01, 2.3040e+00],
          [1.4552e+00, 4.4625e+00, 5.3841e+00, 6.3785e+00],
          [2.9103e-01, 1.2369e+00, 1.2611e+00, 3.2499e+00],
          [1.0429e+00, 1.6249e+00, 2.2555e+00, 2.4980e+00]],

         ...,

         [[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00]],

         [[2.1827

In [15]:
difference = abs( save_output.outputs[9][0] - output_recovered )
print(difference.mean())  ## It should be small, e.g.,2.3 in my trainned model

tensor(2.9704e-07, device='cuda:0', grad_fn=<MeanBackward0>)
